TORONTO NEIGHBORHOODS

Let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install lxml

!pip install geopy

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

1. Download and Explore Dataset

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = requests.get(url)
print(url)
print(toronto_data.status_code)
print(toronto_data.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sat, 10 Aug 2019 14:27:57 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '15024', 'Connection': 'keep-alive', 'Server': 'mw1254.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Content-language': 'en', 'Last-Modified': 'Fri, 26 Jul 2019 12:51:09 GMT', 'Backend-Timing': 'D=104432 t=1565355068991518', 'Vary': 'Accept-Encoding,Cookie,Authorization,X-Seven', 'Content-Encoding': 'gzip', 'X-Varnish': '357225865 28580281, 939511085 343853697', 'Age': '92207', 'X-Cache': 'cp1081 hit/5, cp1085 hit/50', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=10-Aug-2019;Path=/;HttpOnly;secure;Expires=Wed, 11 Sep 2019 12

Let's get the table with the data and assign the rows to the table and assign the "Not assigned" to buroughs - Cleaning

In [3]:
soup = BeautifulSoup(toronto_data.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    Postcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    toronto_data = row[2].text.rstrip()
    if borough != 'Not assigned':
        if toronto_data == 'Not assigned':
            toronto_data = borough
        lst.append([Postcode, borough, toronto_data])
        



Convert to dataframe

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(lst, columns=cols)
print(toronto_data.shape)
toronto_data[toronto_data.duplicated(['PostalCode'], keep=False)] #- this would have shown the duplicate PostalCodes

(211, 3)


,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
11,M4B,East York,Woodbine Gardens
12,M4B,East York,Parkview Hill
13,M5B,Downtown Toronto,Ryerson
14,M5B,Downtown Toronto,Garden District


Use groupby function to arrange the dataframe and keep the first Borough

In [11]:
toronto_data = toronto_data.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

Check if it worked by picking a column

In [12]:
toronto_data.loc[toronto_data['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Check the shape of the new ataframe

In [13]:
toronto_data.shape

(103, 3)

Download the Geospartial data and rename the Postal Code

In [ ]:
toronto_geo = pd.read_csv("https://cocl.us/Geospatial_data")
toronto_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Merge the old data frame with the new one

In [ ]:
toronto_df = pd.merge(toronto_data, toronto_geo, on="PostalCode", how='left')
toronto_df

Read and load the downloaded 

In [ ]:
# load the coordinates from the Geospatial_data
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.head()

Geolocate Toronto

In [ ]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Create a map of Toront with neighborhoods superimposed on top

In [ ]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

Test it on Etobicoke

In [ ]:
Etobicoke_data = toronto_df[toronto_df['Borough'] == 'Etobicoke'].reset_index(drop=True)
Etobicoke_data.head()

Get the geolocator of Etobicoke

In [ ]:
address = 'Etobicoke, ET'

geolocator = Nominatim(user_agent="et_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

Get the map of Etobicoke

In [ ]:
# create map of Manhattan using latitude and longitude values
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

In [ ]:
Downtown_Toronto_df = toronto_df[toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_df.head()                                         #M5H

In [ ]:
address = 'Downtown_Toronto, DT'

geolocator = Nominatim(user_agent="et_explorer")
location = geolocator.geocode(address)
if location:
    latitude = location.latitude
    longitude = location.longitude
print('The geograpical coordinate of Downtown_Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_df['Latitude'], Downtown_Toronto_df['Longitude'], Downtown_Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

Get the Foursquare credentials and print them

In [ ]:
CLIENT_ID = 'xxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

I will focus on Etobicoke

In [ ]:
toronto_df.set_index('PostalCode', inplace = True) 
neighborhood_latitude = toronto_df.loc['M9V']['Latitude']
neighborhood_longitude = toronto_df.loc['M9V']['Longitude']

In [ ]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [ ]:
venues = results['response']['groups'][0]['items']
venues

In [ ]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

Clean the json and structure it into a pandas dataframe.

In [ ]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


Use geopy library to get the latitude and longitude values of Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

And how many venues were returned by Foursquare

In [ ]:
toronto_df

In [ ]:
venues = getNearbyVenues(names=toronto_df['Borough'],latitudes=toronto_df['Latitude'],longitudes=toronto_df['Longitude'])

In [ ]:
print(venues.shape)
venues.head()

Group and count the venues

In [ ]:
venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

One hot encoding to teh dataframe of the categories of the different venues

In [ ]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

In [ ]:
onehot.shape

In [ ]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

In [ ]:
grouped.shape

In [ ]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Returnthe number of common venues and sort the rows by categories

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create columns according to number of clusters and a new dataframe

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Set number of clusters

In [ ]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Add  clustering labels andmerge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

In [ ]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!